<a href="https://colab.research.google.com/github/aaoiii/2024-ESAA-OB/blob/main/5%EC%A3%BC%EC%B0%A8_%ED%8C%8C%EB%A8%B8%EC%99%84_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8. 텍스트 분석

NLP : 머신이 인간의 언어를 이해하고 해석하는 것에 중점
- 텍스트 분석을 향상하게 하는 기반 기술
- 발전 근간에는 머신러닝 존재

텍스트 마이닝 (텍스트 분석) : 비정형 텍스트에서 의미있는 정보를 추출하는 것에 중점
- 비즈니스 인텔리전스나 예측 분석 등의 분석 작업 주로 수행


- 텍스트 분류
- 감성 분석
- 텍스트 요약
- 텍스트 군집화와 유사도 측정


## 01 텍스트 분석 이해
- 비정형 데이터인 텍스트 분석
- 피처 벡터화 / 피터 추출 : 텍스트를 word 기반의 다수의 피처로 추출하고 피처에 단어 빈도수와 같은 숫자 값을 부여하여 텍스트를 벡터값으로 표현
  - BOW(Bag of Words), Word2Vec


<텍스트 분석 수행 프로세스>
1. 텍스트 사전 준비작업 (텍스트 전처리)
  - 클렌징, 대소문자 변경, 특수문자 삭제, 단어 토큰화, 어근 추출, 텍스트 정규화

2. 피처 벡터화 / 추출 : 피처를 추출하고 벡터값 할당
  - BOW (Count 기반, TF-IDF 기반 벡터화) , Word2Vec

3. ML 모델 수립 및 학습/예측 평가


### 파이썬 기반의 NLP, 텍스트 분석 패키지
NLTK (대표적) , Genism (토픽 모델링 분야) , SpaCy (뛰어난 성능, NLP 애플리케이션에서 사용 증가)

- 사이킷런은 어근처리같은 NLP 특화 라이브러리는 없다

## 02 텍스트 사전 준비 작업 (텍스트 전처리-텍스트 정규화)

- 클렌징
- 토큰화
- 필터링/스톱워드 제거/철자 수정
- Stemming
- Lemmatization



### [1] 클렌징 : 불필요한 문자, 기호 제거
### [2] 토큰화 : 문장토큰화 / 단어토큰화
**1) 문장 토큰화**
- 문장의 마지막을 뜻하는 기호에 따라 분리
- sent_tokenize

In [2]:
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

text_sample = 'The Matrix is everywhere its all around us, here even in this room. \
                You can see it out your window or on your television. \
                You feel it when you go to work, or go to church or pay your taxes.'
sentences = sent_tokenize(text = text_sample)

print(type(sentences), len(sentences))
print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<class 'list'> 3
['The Matrix is everywhere its all around us, here even in this room.', 'You can see it out your window or on your television.', 'You feel it when you go to work, or go to church or pay your taxes.']


**2) 단어 토큰화**
- 문장을 단어로 토큰화 (공백, 콤마, 마침표, 개행문자로 분리)
- 정규 표현식을 이용해 다양한 유형으로 토큰화 수행 가능
- Bag of Words 와 같이 단어의 순서가 중요하지 않은 경우 문장 토큰화를 사용하지 않고 단어 토큰화만 사용해도 된다
- word_tokenize()

In [3]:
from nltk import word_tokenize

sentence = 'The Matrix is everywhere its all around us, here even in this room.'
words = word_tokenize(sentence)
print(type(words), len(words))
print(words)

<class 'list'> 15
['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


**3) sent_tokenize와 word_tokenize를 조합**

In [4]:
from nltk import word_tokenize, sent_tokenize

# 여러 개의 문장으로 된 입력 데이터를 문장별로 단어 토큰화하게 만드는 함수 생성
def tokenize_text(text):

    # 문장별로 분리 토큰
    sentences = sent_tokenize(text)

    # 분리된 문장별 단어 토큰화
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

# 여러 문장에 대해 문장별 단어 토큰화 수행
word_tokens = tokenize_text(text_sample)
print(type(word_tokens), len(word_tokens))
print(word_tokens)

<class 'list'> 3
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


문맥적 의미가 무시된다 => n-gram

n-gram : 연속된 n개의 단어를 하나의 토큰화 단위로 분리
- n개 단어크기 윈도우를 만들어 문장의 처음부터 오른쪽으로 움직이면서 토큰화 수행

### [3] 스톱워드 제거

스톱워드 : 분석에 의미가 없는 단어

In [5]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print('영어 stop words 개수 : ', len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

영어 stop words 개수 :  179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [7]:
# 위의 word_tokens 리스트에 대해서 stopwords를 필터링으로 제거해보기


import nltk

stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []

# 위 예제에서 3개의 문장별로 얻은 word_tokens list에 대해 스톱워드를 제거하는 반복문
for sentence in word_tokens :
    filtered_words = []
    # 개별 문장별로 토큰화된 문장 list에 대해 스톱 워드를 제거하는 반복문
    for word in sentence :
        # 소문자로 모두 변환한다.
        word = word.lower()
        # 토큰화된 개별 단어가 스톱워드의 단어에 포함되지 않으면 word_tokens에 추가
        if word not in stopwords :
            filtered_words.append(word)
    all_tokens.append(filtered_words)

print(all_tokens)

[['matrix', 'everywhere', 'around', 'us', ',', 'even', 'room', '.'], ['see', 'window', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


is, this 같은 스톱워드 제거

### [4] Stemming 과 Lemmatization
- 문법적 또는 의미적으로 변화하는 단어의 원형을 찾는 것

**Lemmatization**이 Stemming보다 정교하며 의미론적인 기반에서 단어의 원형을 찾는다
- 품사와 같은 문법적인 요소와 의미적인 부분을 감안해 정확한 철자로 된 어근 단어를 찾아준다
- WordNetLemmatizer


**Stemming**은 원형 단어로 변환 시 일반적인 방법을 적용하거나 더 단순화된 방법을 적용해 원래 단어에서 일부 철자가 훼손된 어근 단어를 추출하는 경향이 있다
- Porter, Lancaster, Snoball Stemmer

< LancasterStemmer >
- 진행형, 3인칭 단수, 과거형에 따른 동사, 비교, 최상에 따른 형용사의 변화에 따라 더 단순하게 원형 단어를 찾아준다

In [8]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem('working'), stemmer.stem('works'), stemmer.stem('worked'))
print(stemmer.stem('amusing'), stemmer.stem('amuses'), stemmer.stem('amused'))
print(stemmer.stem('happier'), stemmer.stem('happiest'))
print(stemmer.stem('fancier'), stemmer.stem('fanciest'))

work work work
amus amus amus
happy happiest
fant fanciest


< WordNetLemmatizer >
- 정확한 원형단어추출을 위해 단어의 품사를 입력해줘야한다
- 동사 : v, 형용사 : a

In [9]:
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')

lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing', 'v'), lemma.lemmatize('amuses', 'v'), lemma.lemmatize('amused', 'v'))
print(lemma.lemmatize('happier', 'a'), lemma.lemmatize('happiest', 'a'))
print(lemma.lemmatize('fancier', 'a'), lemma.lemmatize('fanciest', 'a'))

[nltk_data] Downloading package wordnet to /root/nltk_data...


amuse amuse amuse
happy happy
fancy fancy


## 03 Bag of Words - BOW

- 단어의 문맥이나 순서를 무시하고 일괄적으로 단어에 대해 빈도 값을 부여해 피처 값 추출

문장 2개가 있을 때..
1. 문장1과 2에 있는 모든 단어에서 중복을 제거하고 각 단어를 칼럼 형태로 나열하고 각 단어에 고유의 인덱스를 부여

2. 개별 문장에서 해당 단어가 나타나는 횟수를 각 단어에 기재

단점
- 문맥 의미 반영 부족
- 희소 행렬 문제 (희소성, 희소행렬)
  - 대규모의 칼럼에서 구성된 행렬에서 대부분의 값이 0인 행렬

### BOW 피처 벡터화
- 각 문서의 텍스트를 단어로 추출해 피처로 할당, 각 단어의 발생 빈도와 같은 값을 피처에 부여하여 문서를 벡터로 만드는 기법
- 모든 단어를 칼럼 형태로 나열하고 각 문서에서 해당 단어의 횟수나 정규화된 빈도를 값으로 부여하는 데이터 세트 모델로 변경
- M개의 텍스트 문서, N개의 단어 => M*N 단어피처로 이뤄진 행렬


1) 카운트 기반 벡터화
- 해당 단어 나타나는 횟수
- 카운트 높으면 중요한 단어

2) TF-IDF (Term Frequency - Inverse Document Frequency) 기반 벡터화
- 자주 나타나는 단어에 높은 가중치, 전반적으로 자주 나타나는 단어에 대해서는 패널티 부여

### 사이킷런의 Count 및 TF-IDF 벡터화 구현 : CountVectorizer, TfidfVectorizer

**CountVectorizer**
- 카운트 기반 벡터화
- 소문자 일괄 변환, 토큰화, 스톱 워드필터링 등의 텍스트 전처리 함께 수행
- fit transform


파라미터
- max_df : 너무 높은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
- min_df : 너무 낮은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
- max_features : 추출하는 피처의 개수를 제한
- stop_words : 'english'로 지정하면 영어의 스톱워드로 지정된 단어는 추출에서 제외
- n_gram_range : tuple 형태로 (범위 최솟값, 범위 최댓값)을 지정
- analyzer : 피처 추출을 수행한 단위, 디폴트는 word
- token_pattern : 토큰화를 수행하는 정규 표현식 패턴 지정
-tokenizer : 토큰화를 별도의 커스텀 함수로 이용시 적용

사전 데이터 가공 => 토큰화 => 텍스트 정규화 => 피처 벡터화

**TfidfVectorizer**

파라미터, 변환 방법 동일

### BOW 벡터화를 위한 희소 행렬

COO 형식, CSR 형식 : 희소행렬이 적은 메모리 공간을 차지할 수 있도로 변환

### 희소행렬 - COO 형식
0이 아닌 데이터만 별도의 데이터 배열에 저장하고 데이터가 가리키는 행과 열의 위치를 별도의 배열로 저장
- Scipy 사이파이 이용 - sparse 패키지

In [10]:
import numpy as np

dense = np.array([[3,0,1], [0,2,0]])

In [11]:
from scipy import sparse

# 0이 아닌 데이터 추출
data = np.array([3,1,2])

# 행 위치와 열 위치를 각각 배열로 생성
row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

# sparse 패키지의 coo_matrix를 이용해 COO 형식으로 희소 행렬 생성
sparse_coo = sparse.coo_matrix((data, (row_pos, col_pos)))

In [12]:
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

### 희소행렬 - CSR 형식
COO 형식의 반복적인 위치 데이터를 사용해야하는 문제점 해결
- 행 위치 배열 내에 있는 고유한 값의 시작 위치만 다시 별도의 위치 배열로 가지는 변환 방식

In [13]:
from scipy import sparse

dense = np.array([[0,0,1,0,0,5],
                  [1,4,0,3,2,5],
                  [0,6,0,3,0,0],
                  [2,0,0,0,0,0],
                  [0,0,0,7,0,8],
                  [1,0,0,0,0,0]

])

# 0이 아닌 데이터 추출
data2 = np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])

# 행 위치와 열 위치를 각각 array로 생성
row_pos = np.array([0,0,1,1,1,1,1,2,2,3,4,4,5])
col_pos = np.array([2,5,0,1,3,4,5,1,3,0,3,5,0])

# COO 형식으로 변환
sparse_coo = sparse.coo_matrix((data2, (row_pos, col_pos)))

# 행 위치 배열의 고유한 값의 시작 위치 인덱스를 배열로 생성
row_pos_ind = np.array([0,2,7,9,10,12,13])

# CSR 형식으로 변환
sparse_csr = sparse.csr_matrix((data2, col_pos, row_pos_ind))

print('COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_coo.toarray())
print('CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_csr.toarray())

COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]
CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


In [14]:
dense3= np.array([[0,0,1,0,0,5],
                  [1,4,0,3,2,5],
                  [0,6,0,3,0,0],
                  [2,0,0,0,0,0],
                  [0,0,0,7,0,8],
                  [1,0,0,0,0,0]

])

coo = sparse.coo_matrix(dense3)
csr = sparse.csr_matrix(dense3)